# Sample Jupyter ProbLog notebook
This is a simple [python wrapper kernel](https://jupyter-client.readthedocs.io/en/latest/wrapperkernels.html) for [ProbLog](https://dtai.cs.kuleuven.be/problog/).

## Supported modes
Currently, only the default problog mode (inference) is supported.

## Cell Ids
If the first non-empty line in a cell starts with `%#`, whatever follows is considered the id of the cell.

Modifying a cell with an id and re-evaluating will emulate an update of the cell - all the previously evaluated static-clauses are removed from the program, and the contents of the cell is added again.
Side-effects will *not* be undone.

In [1]:
%# 1

0.2::foo(2); 0.3::foo(3); 0.5::foo(5):- bar(Z).

In [2]:
%# 2
bar(a).
0.7::bar(b).

evidence(\+bar(b)).

query(foo(X)).

foo(5): 0.500000
foo(3): 0.300000
foo(2): 0.200000